# Functors - Objects with a map function

## Example 1: Arrays

In [26]:
class Array:
    def __init__(self, *args):
        self.values = [*args]
    def map(self, func):
        return Array(*list(map(func, self.values)))
    def __str__(self):
        return self.values.__str__()

arr = Array(1,2,3)
print(arr)
arr2 = arr.map(lambda v: v*2)
print(arr2)

[1, 2, 3]
[2, 4, 6]


In [27]:
class Functor:
    def __init__(self, value):
        self.value = value
    def map(self, func):
        return Functor(func(self.value))
    def __str__(self):
        return "Functor(%s)" % self.value

In [28]:
Five = Functor(5)
print(Five)
Ten = Five.map(lambda v: v*2)
print(Ten)

Functor(5)
Functor(10)


In [29]:
BadTen = Five.map(lambda v: Functor(v*2))
print(BadTen)

Functor(Functor(10))


# Monad - Objects with flat_map function

In [30]:
import asyncio
import requests
from IPython.display import Image 

def random_dog():
    response = requests.get("https://dog.ceo/api/breeds/image/random")
    return response.json()

def fetch_image(url):
    return Image(url=url)
    
loop = asyncio.get_running_loop()
result = loop.run_in_executor(None, random_dog)
print("waiting for random dog...")
url = (await result)["message"]
image = loop.run_in_executor(None, lambda: fetch_image(url))
print("downloading dog...")
display(await image)

waiting for random dog...
downloading dog...


In [31]:
import asyncio
import requests
from IPython.display import Image 

def get_url(url):
    print("requesting %s" % url)
    return  requests.get(url)

def fetch_image(url):
    print("generating image %s" % url)
    return Image(url=url)

class Vow:
    def __init__(self, value=None):
        self.value = value
    def maybe_wrap(value):
        if type(value) == Vow:
            return value
        return Vow(value)
        
    def flat_map(self, func):
        loop = asyncio.get_running_loop()
        # If value is a Vow, simply chain the func
        if type(self.value) == Vow:
            return Vow.maybe_wrap(self.value.flat_map(func))
        # If value is a Future value, then once it will be done execute func on the value
        if asyncio.isfuture(self.value):
            new_future = loop.create_future()
            self.value.add_done_callback(lambda _: new_future.set_result(Vow.maybe_wrap(self.value.result()).flat_map(func)))
            return Vow(new_future)
        # If it is a simple value then tell the event loop to execute it later
        return Vow(loop.run_in_executor(None, lambda: func(self.value)))
    def when(self, func):
        return self.flat_map(func)

    
Vow("https://dog.ceo/api/breeds/image/random") \
    .flat_map(get_url)\
    .flat_map(lambda response: Vow(response.json())) \
    .flat_map(lambda json: json["message"]) \
    .flat_map(fetch_image) \
    .flat_map(display)

print("")

requesting https://dog.ceo/api/breeds/image/random

generating image https://images.dog.ceo/breeds/eskimo/n02109961_9973.jpg


In [32]:
Vow("https://dog.ceo/api/breeds/image/random") \
    .when(get_url)\
    .when(lambda response: Vow(response.json())) \
    .when(lambda json: json["message"]) \
    .when(fetch_image) \
    .when(display)
print("")

requesting https://dog.ceo/api/breeds/image/random

generating image https://images.dog.ceo/breeds/hound-afghan/n02088094_13145.jpg
